# Analysis Notebook

In [1]:
from utils import *
import os

## JG Function

In [7]:
def run_jg2(weak_answer:str, SPEC_NAME, signal_name, i, toret='string'):
    import paramiko
    import re
    from dotenv import load_dotenv
    load_dotenv()

    # SSH server details
    hostname =  os.environ.get(f"HOST_NAME")
    port = 22
    username = os.environ.get("USERNAME")
    password = os.environ.get("PASSWORD")

    # print(hostname, username, password)

    # Create SSH client
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    local_log_path = "temp.txt"

    ROOT = "/storage/ckarfa/hpdmc/"
    if SPEC_NAME == "hpdmc":
        remote_sv_file_path = ROOT + "sva/hpdmc/hpdmc.sv"
        remote_log_path = ROOT + "output.txt"
        sv_template = HPDMC_sv_template
        JG_COMMAND = """
            csh -c '
            source cad_cshrc
            cd hpdmc
            jg -batch hpdmc.tcl > output.txt
            '
        """
    elif SPEC_NAME == "i2c":
        remote_sv_file_path = ROOT + "sva/i2c/i2c.sv"
        remote_log_path = ROOT + "output.txt"
        sv_template = I2C_sv_template
        JG_COMMAND = """
            csh -c '
            source cad_cshrc
            cd hpdmc
            jg -batch i2c.tcl > output.txt
            '
        """
    elif SPEC_NAME == 'rv':
        remote_sv_file_path = ROOT + 'sva/rv/rv.sv'
        remote_log_path = ROOT + 'output.txt'
        sv_template = RV_sv_template
        JG_COMMAND = '''
            csh -c '
            source cad_cshrc
            cd hpdmc
            jg -batch rv.tcl > output.txt
            '
        '''

    def extract_svas(sv_code: str, parameter_string: str):
        # single_line_pattern = re.compile(r'assert property \(@\(.*?\) (.*?)\);', re.DOTALL)
        # Extract only valid single-line SVAs (ignoring empty property references)
        # single_line_svas = [f"assert property (@(posedge sys_clk) {sva});" for sva in single_line_pattern.findall(sv_code)]

        # Updated regex to strictly capture assertions with `@(...)`
        single_line_pattern = re.compile(r'assert property\s*\(@\([^)]*\)\s*.*?\);')
        # Extract only valid single-line SVAs while keeping their clock cycle
        single_line_svas = single_line_pattern.findall(sv_code)

        multi_line_pattern = re.compile(
            r'(property\s+\w+;.*?endproperty\s+assert property \(\w+\);)', 
            re.DOTALL
        )        
        # Extract full multi-line SVAs as a whole block
        multi_line_svas = multi_line_pattern.findall(sv_code)

        # Return only fully valid SVAs by appending with parameters
        ret = []
        for item in (single_line_svas + multi_line_svas):
            ret.append(parameter_string + item)

        return ret

    def extract_assertions(weak_answer):
        # Extract the SystemVerilog block
        code_block_matches = re.findall(r"```systemverilog(.*?)```", weak_answer, re.DOTALL)
        if not code_block_matches:
            return []
        
        code_block = code_block_matches[-1].strip()

        # Extract parameters (lines that start with "parameter")
        parameter_pattern = r"parameter\s+.*?;"
        parameters = re.findall(parameter_pattern, code_block, re.MULTILINE)
        parameter_string1 = ""
        for parameter in parameters:
            parameter_string1 += parameter + "\n"

        parameter_pattern = r"localparam\s+.*?;"
        parameters = re.findall(parameter_pattern, code_block, re.MULTILINE)
        parameter_string2 = ""
        for parameter in parameters:
            parameter_string2 += parameter + "\n"

        return extract_svas(code_block, parameter_string1 + parameter_string2)

    assertions = extract_assertions(weak_answer)

    # for item in assertions:
    #     print(item)
    #     print("-" * 80)

    def modify_template(assertion_entry):
        """Insert parameters and assertion into the SystemVerilog template."""
        return sv_template.format(properties=f"{assertion_entry}")

    def execute_ssh_commands(ssh):
        """Runs the required commands in a single SSH session."""
        command = JG_COMMAND
        stdin, stdout, stderr = ssh.exec_command(command)
        stdout.channel.recv_exit_status()  # Wait for execution

    try:
        ssh.connect(hostname, port, username, password)
        sftp = ssh.open_sftp()
        results = {}
        from time import time
        start = time()

        for assertion_entry in assertions:
            modified_sv = modify_template(assertion_entry)

            # print("Getting Feedback from JG - Starting File Upload")

            # Upload the new SystemVerilog file
            with sftp.file(remote_sv_file_path, "w") as f:
                f.write(modified_sv)

            # print("Getting Feedback from JG - File Upload Successful")

            execute_ssh_commands(ssh)

            # print("Getting Feedback from JG - Command Execution Successful. Downloading Logs")

            sftp.get(remote_log_path, local_log_path)

            with open(local_log_path, "r") as log_file:
                log_content = log_file.read()

            # Store the log output mapped to the assertion
            results[assertion_entry] = log_content

        # Close connections
        sftp.close()
        ssh.close()
        end = time()

        print("Total Time Taken to get JG Feedback: ", end - start)

        import pickle 

        with open(f'JasperGold_Logs/{SPEC_NAME}/{signal_name}_{i}.pkl', 'wb') as f:
            pickle.dump(results, f)
            
    except Exception as e:
        print(f"Error: {e}")

    # Now I only need to pass on the assertions that gave syntax error here and also check if I need to continue
    toContinue = False

    # This values are just for logging and maybe removed if needed
    CompleteCorrect = 0
    SyntaxCorrectSemanticsWrong = 0
    SyntaxWrong = 0

    for key, value in results.items():
        if "=\nSUMMARY\n=" in value:
            # Look for the "- cex                       : " pattern
            match = re.search(r"- cex\s+:\s+(\d)", value)
            
            if match:
                cex_value = int(match.group(1))  # Extract number (0 or 1)
                if cex_value == 0:
                    CompleteCorrect += 1
                else:
                    # semantically not correct according to the current RTL
                    toContinue = True
                    SyntaxCorrectSemanticsWrong += 1

            results[key] = "No Syntax Errors Found. Ensure semantics are correct wrt Specification"
        else:
            toContinue = True # Syntactically wrong assertions
            SyntaxWrong += 1

    # print(f"Results Summary:\n\tSyntax Wrong: {SyntaxWrong}\n\tSyntax Correct Semantics Wrong: {SyntaxCorrectSemanticsWrong}\n\tComplete Correct: {CompleteCorrect}")

    if toret == 'string':
        return str(results), SyntaxWrong, SyntaxCorrectSemanticsWrong, CompleteCorrect  
    elif toret == 'dict':
        return results, toContinue


## i2c Analysis

In [3]:
import pandas as pd

In [16]:
df = pd.read_csv('./rvFinal.csv')

In [17]:
df

,spec_name,signal_name,information,output,COMBINER
0,rv,NaN,Here is the mapping information:\r\n[clk_i]: 1...,NaN,NaN
1,rv,clk_i,\r\n[Signal name]: clk_i\r\n[Description]:\r\n...,"<think>\nOkay, let's tackle generating the SVA...","<think>\nOkay, let me tackle this problem step..."
2,rv,rst_ni,\r\n[Signal name]: rst_ni\r\n[Description]:\r\...,"<think>\nOkay, let's tackle generating the SVA...","<think>\nOkay, let's tackle this. The user wan..."
3,rv,active,\r\n[Signal name]: active \r\n[Description]: ...,"<think>\nOkay, let's tackle generating the SVA...","<think>\nOkay, let's tackle this step by step...."
4,rv,prescaler,\r\n[Signal name]: prescaler \r\n[Description...,"<think>\nOkay, let's tackle generating the SVA...","<think>\nOkay, so I need to extract all unique..."
5,rv,step,\r\n[Signal name]: step \r\n[Description]: \...,"<think>\nOkay, let's tackle generating the SVA...","<think>\nOkay, let's tackle this problem. The ..."
6,rv,mtime,\r\n[Signal name]: mtime \r\n[Description]: ...,"<think>\nOkay, let's tackle generating the SVA...","<think>\nOkay, let's tackle this problem. The ..."
7,rv,mtimecmp,\r\n[Signal name]: mtimecmp \r\n[Description]...,"<think>\nOkay, let's tackle generating the SVA...","<think>\nOkay, I need to extract all unique an..."
8,rv,tick,\r\n[Signal name]: tick \r\n[Description]: \...,"<think>\nOkay, let's tackle generating the SVA...","<think>\nOkay, let's tackle this step by step...."
9,rv,mtime_d,\r\n[Signal name]: mtime_d \r\n[Description]:...,"<think>\nOkay, let's tackle generating the SVA...","<think>\nOkay, I need to extract all unique an..."


In [18]:
from tabulate import tabulate

table_data = [
    ["Signal Name", "Syntax Wrong", "Syntax Wrong, Semantics Correct", "Complete Correct"]
]

# Populate the rows with data
for index, row in df.iterrows():
    if index == 0:
        continue

    with open(f"{row['signal_name']}.txt", 'r') as f:
        text = f.read()

    _, a, b, c = run_jg2(text, row['spec_name'], row['signal_name'], 'Analysis')
    print(f"{[row['signal_name'], a, b, c]}")
    
    table_data.append([row['signal_name'], a, b, c])

Total Time Taken to get JG Feedback:  21.97671866416931
['clk_i', 0, 1, 2]
Total Time Taken to get JG Feedback:  113.82078504562378
['rst_ni', 0, 10, 4]
Total Time Taken to get JG Feedback:  55.32325100898743
['active', 0, 0, 5]
Total Time Taken to get JG Feedback:  79.90055394172668
['prescaler', 3, 1, 5]
Total Time Taken to get JG Feedback:  28.95219373703003
['step', 0, 0, 3]
Total Time Taken to get JG Feedback:  42.38502597808838
['mtime', 0, 0, 4]
Total Time Taken to get JG Feedback:  42.18297243118286
['mtimecmp', 0, 0, 4]
Total Time Taken to get JG Feedback:  91.18821334838867
['tick', 0, 2, 8]
Total Time Taken to get JG Feedback:  102.37910771369934
['mtime_d', 0, 4, 8]
Total Time Taken to get JG Feedback:  28.168726921081543
['intr', 5, 1, 0]


In [19]:
# Print the formatted table
print("Analysis Table after Automated Combiner for rv Specification:")
print(tabulate(table_data, headers="firstrow", tablefmt="grid"))

Analysis Table after Automated Combiner for rv Specification:
+---------------+----------------+-----------------------------------+--------------------+
| Signal Name   |   Syntax Wrong |   Syntax Wrong, Semantics Correct |   Complete Correct |
+===============+================+===================================+====================+
| clk_i         |              0 |                                 1 |                  2 |
+---------------+----------------+-----------------------------------+--------------------+
| rst_ni        |              0 |                                10 |                  4 |
+---------------+----------------+-----------------------------------+--------------------+
| active        |              0 |                                 0 |                  5 |
+---------------+----------------+-----------------------------------+--------------------+
| prescaler     |              3 |                                 1 |                  5 |
+---------------+-

In [20]:
table_data

[['Signal Name',
  'Syntax Wrong',
  'Syntax Wrong, Semantics Correct',
  'Complete Correct'],
 ['clk_i', 0, 1, 2],
 ['rst_ni', 0, 10, 4],
 ['active', 0, 0, 5],
 ['prescaler', 3, 1, 5],
 ['step', 0, 0, 3],
 ['mtime', 0, 0, 4],
 ['mtimecmp', 0, 0, 4],
 ['tick', 0, 2, 8],
 ['mtime_d', 0, 4, 8],
 ['intr', 5, 1, 0]]

In [21]:
df = pd.DataFrame(table_data[1:], columns=table_data[0])
print(df)

  Signal Name  Syntax Wrong  Syntax Wrong, Semantics Correct  Complete Correct
0       clk_i             0                                1                 2
1      rst_ni             0                               10                 4
2      active             0                                0                 5
3   prescaler             3                                1                 5
4        step             0                                0                 3
5       mtime             0                                0                 4
6    mtimecmp             0                                0                 4
7        tick             0                                2                 8
8     mtime_d             0                                4                 8
9        intr             5                                1                 0


In [22]:
df.to_csv('rvAnalysis.csv', index=False)